# Speech Recognition

## Imports

In [ ]:
import os
import pickle
import librosa
import pandas as pd
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

from IPython.display import Audio

import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import backend as K

## Data Preparation

In [ ]:
rows = []
parent_dir = "../SWH-05-20101106"
files = os.listdir(parent_dir)
for f in files:
    audio, fs = librosa.load(f"{parent_dir}/{f}")
    filename = f.split('.')[0]
    row = {'filename': filename, 'audio': audio}
    rows.append(row)
rows[:5]

[{'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part102',
  'audio': array([-0.01096754, -0.01230842, -0.01015999, ..., -0.21667908,
         -0.20379573, -0.11009098], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part107',
  'audio': array([-0.00262849, -0.00256155, -0.00178459, ..., -0.2567303 ,
         -0.21261317,  0.        ], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part12',
  'audio': array([-0.00823285, -0.00249539, -0.00311783, ..., -0.18402188,
         -0.19362031, -0.11912253], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part64',
  'audio': array([ 0.00796661,  0.0098575 ,  0.0089713 , ...,  0.08651416,
          0.04152827, -0.00214096], dtype=float32)},
 {'filename': 'SWH-05-20101106_16k-emission_swahili_05h30_-_06h00_tu_20101106_part68',
  'audio': array([-0.0083479 , -0.0101786 , -0.0102

In [ ]:
sample_audios = []
for row in rows:
    audio = row['audio']
    sample_audios.append(audio)
sample_audios[:5]

[array([-0.01096754, -0.01230842, -0.01015999, ..., -0.21667908,
        -0.20379573, -0.11009098], dtype=float32),
 array([-0.00262849, -0.00256155, -0.00178459, ..., -0.2567303 ,
        -0.21261317,  0.        ], dtype=float32),
 array([-0.00823285, -0.00249539, -0.00311783, ..., -0.18402188,
        -0.19362031, -0.11912253], dtype=float32),
 array([ 0.00796661,  0.0098575 ,  0.0089713 , ...,  0.08651416,
         0.04152827, -0.00214096], dtype=float32),
 array([-0.0083479 , -0.0101786 , -0.01026751, ...,  0.0113914 ,
         0.02161873,  0.        ], dtype=float32)]

In [ ]:
meta_df = pd.read_csv('../metadata.csv')

In [ ]:
meta_df.head()

,filename,transcription,filepath,sample_rate,duration
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.14
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.10
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.65
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,3.90
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya,SWH-05-20101106/SWH-05-20101106_16k-emission_s...,16000,2.94


In [ ]:
meta_df['sample_rate'].value_counts()

16000    10180
Name: sample_rate, dtype: int64

In [ ]:
meta_df.columns.to_list()

['filename', 'transcription', 'filepath', 'sample_rate', 'duration']

In [ ]:
txts = []
for row in rows:
    filename = row['filename']
    filter = meta_df[meta_df['filename'] == filename]
    txt = filter[['transcription']].values
    txts.append(txt)

txts[:5]

[array([['juma hili bara la afrika limeshuhudia raia wa nchi za niger']],
       dtype=object),
 array([['na rais aliyetangulia henry konan berdi']], dtype=object),
 array([['baada ya kushinda katika uchaguzi mkuu wa taifa hilo']],
       dtype=object),
 array([['siku ya jumatano maharamia hao wa kisomali']], dtype=object),
 array([['pamoja na abiria ishirini wakiwemo raia wa madagascar']],
       dtype=object)]

In [ ]:
txts = np.array(txts).reshape(-1)

In [ ]:
txts[:5]

array(['juma hili bara la afrika limeshuhudia raia wa nchi za niger',
       'na rais aliyetangulia henry konan berdi',
       'baada ya kushinda katika uchaguzi mkuu wa taifa hilo',
       'siku ya jumatano maharamia hao wa kisomali',
       'pamoja na abiria ishirini wakiwemo raia wa madagascar'],
      dtype=object)

In [ ]:
clean_txts = []
alphabets = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'.split()
for txt in txts:
    clean_txt = []
    for c in txt:
        if c not in alphabets and c != ' ':
            continue
        clean_txt.append(c)
    clean_txt = ''.join(clean_txt)
    clean_txts.append(clean_txt)

In [ ]:
clean_txts[:5]

['juma hili bara la afrika limeshuhudia raia wa nchi za niger',
 'na rais aliyetangulia henry konan berdi',
 'baada ya kushinda katika uchaguzi mkuu wa taifa hilo',
 'siku ya jumatano maharamia hao wa kisomali',
 'pamoja na abiria ishirini wakiwemo raia wa madagascar']

In [ ]:
'' in clean_txts

True

In [ ]:
df = pd.DataFrame(clean_txts)
df.columns = ['texts']
df.head()

,texts
0,juma hili bara la afrika limeshuhudia raia wa ...
1,na rais aliyetangulia henry konan berdi
2,baada ya kushinda katika uchaguzi mkuu wa taif...
3,siku ya jumatano maharamia hao wa kisomali
4,pamoja na abiria ishirini wakiwemo raia wa mad...


In [ ]:
idxs = df[df['texts'] == ''].index
idxs

Int64Index([51, 161, 190], dtype='int64')

In [ ]:
del clean_txts[idxs[-1]]
del clean_txts[idxs[-2]]
del clean_txts[idxs[-3]]

In [ ]:
'' in clean_txts

False

In [ ]:
del sample_audios[idxs[-1]]
del sample_audios[idxs[-2]]
del sample_audios[idxs[-3]]

## Tokenizer

In [ ]:
def character_dict():
    alphabet = 'a b c d e f g h i j k l m n o p q r s t u v w x y z'
    supported = alphabet.split()

    char_map = {}
    char_map[""] = 0
    char_map["<SPACE>"] = 1
    idx = 2
    for c in supported:
        char_map[c] = idx
        idx += 1
    index_map = {v: k for k, v in char_map.items()}
    return char_map, index_map

In [ ]:
char_map, index_map = character_dict()

In [ ]:
char_map

{'': 0,
 '<SPACE>': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'j': 11,
 'k': 12,
 'l': 13,
 'm': 14,
 'n': 15,
 'o': 16,
 'p': 17,
 'q': 18,
 'r': 19,
 's': 20,
 't': 21,
 'u': 22,
 'v': 23,
 'w': 24,
 'x': 25,
 'y': 26,
 'z': 27}

In [ ]:
def text_to_int_sequence(text):
    """ Convert text to an integer sequence """
    int_sequence = []
    for c in text:
        if c == ' ':
            ch = char_map['<SPACE>']
        elif c in alphabets:
            ch = char_map[c]
        else:
            print(c)
            print('character not found')
            break
        int_sequence.append(ch)
    return np.array(int_sequence)

In [ ]:
def int_sequence_to_text(int_sequence):
    """ Convert an integer sequence to text """
    textch = []
    for c in int_sequence:
        ch = index_map[c]
        textch.append(ch)
    text = ''.join(textch)
    text = text.replace('<SPACE>', ' ')
    return text

## Data Generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, audios, texts, batch_size=32):
        self.audios = audios
        self.texts = texts
        self.batch_size = batch_size
        self.steps = int(len(self.audios) // self.batch_size)
        # self.index = 0
        self.on_epoch_end()

    # def shuffle(self):
    #     np.random.shuffle(self.indexes)

    def __len__(self):
        return self.steps

    def on_epoch_end(self):
        self.indexes = np.arange(self.steps*self.batch_size)
        # np.random.shuffle(self.indexes)

    def data_generation(self, batch_audios, batch_texts):

        longest_audio = max([len(i) for i in batch_audios])
        longest_txt = max([len(i) for i in batch_texts])

        audios          = np.zeros([int(self.batch_size), longest_audio], dtype="float32")
        txts            = np.zeros([int(self.batch_size), longest_txt], dtype="int64")
        audio_length    = np.zeros([int(self.batch_size)], dtype="int64")
        txt_length      = np.zeros([int(self.batch_size)], dtype="int64")

        i = 0
        for audio, txt in zip(batch_audios, batch_texts):

            txt_len = len(txt)

            txt = text_to_int_sequence(txt)
            # print(txts.shape)
            # print(np.array(txt).shape)
            txts[i,: txt_len] = txt

            audio_len = len(audio)

            audios[i, :audio_len] = audio

            audio_length[i] = audio_len
            txt_length[i] = txt_len

            i+=1          
            
        outputs = {'ctc': np.zeros([self.batch_size])}
        inputs = {
                    'the_input':    tf.convert_to_tensor(audios), 
                    'the_labels':   tf.convert_to_tensor(txts), 
                    'input_length': tf.convert_to_tensor(audio_length), 
                    'label_length': tf.convert_to_tensor(txt_length)
                }
        return (inputs, outputs)


    def __getitem__(self, index):
        indexes = self.indexes[int(index*self.batch_size):int((index+1)*self.batch_size)]
    
        batch_audios = [self.audios[int(i)] for i in indexes]
        batch_texts = [self.texts[int(i)] for i in indexes]
        
        return  self.data_generation(batch_audios, batch_texts)

In [ ]:
dg = DataGenerator(sample_audios, clean_txts)

In [ ]:
len(dg)

6

In [ ]:
batch1 = dg[0][0]


In [ ]:
batch1

<tf.Tensor: shape=(32, 112), dtype=int64, numpy=
array([[11, 22, 14, ...,  0,  0,  0],
       [15,  2,  1, ...,  0,  0,  0],
       [ 3,  2,  2, ...,  0,  0,  0],
       ...,
       [12, 10, 21, ...,  0,  0,  0],
       [10, 13,  2, ..., 16, 27, 10],
       [11,  6, 11, ...,  0,  0,  0]])>

## LogMelSpectrogram

In [ ]:
class LogMelSpectrogram(tf.keras.layers.Layer):
    """Compute log-magnitude mel-scaled spectrograms."""

    def __init__(self, sample_rate, fft_size, hop_size, n_mels,
                 f_min=0.0, f_max=None, **kwargs):
        super(LogMelSpectrogram, self).__init__(**kwargs)
        self.sample_rate = sample_rate
        self.fft_size = fft_size
        self.hop_size = hop_size
        self.n_mels = n_mels
        self.f_min = f_min
        self.f_max = f_max if f_max else sample_rate / 2
        self.mel_filterbank = tf.signal.linear_to_mel_weight_matrix(
            num_mel_bins=self.n_mels,
            num_spectrogram_bins=fft_size // 2 + 1,
            sample_rate=self.sample_rate,
            lower_edge_hertz=self.f_min,
            upper_edge_hertz=self.f_max)

    def build(self, input_shape):
        self.non_trainable_weights.append(self.mel_filterbank)
        super(LogMelSpectrogram, self).build(input_shape)

    def call(self, waveforms):
        """Forward pass.
        Parameters
        ----------
        waveforms : tf.Tensor, shape = (None, n_samples)
            A Batch of mono waveforms.
        Returns
        -------
        log_mel_spectrograms : (tf.Tensor), shape = (None, time, freq, ch)
            The corresponding batch of log-mel-spectrograms
        """
        def _tf_log10(x):
            numerator = tf.math.log(x)
            denominator = tf.math.log(tf.constant(10, dtype=numerator.dtype))
            return numerator / denominator

        def power_to_db(magnitude, amin=1e-16, top_db=80.0):
            """
            https://librosa.github.io/librosa/generated/librosa.core.power_to_db.html
            """
            ref_value = tf.reduce_max(magnitude)
            log_spec = 10.0 * _tf_log10(tf.maximum(amin, magnitude))
            log_spec -= 10.0 * _tf_log10(tf.maximum(amin, ref_value))
            log_spec = tf.maximum(log_spec, tf.reduce_max(log_spec) - top_db)

            return log_spec

        spectrograms = tf.signal.stft(waveforms,
                                      frame_length=self.fft_size,
                                      frame_step=self.hop_size,
                                      pad_end=False)

        magnitude_spectrograms = tf.abs(spectrograms)

        mel_spectrograms = tf.matmul(tf.square(magnitude_spectrograms),
                                     self.mel_filterbank)

        log_mel_spectrograms = power_to_db(mel_spectrograms)

        # add channel dimension
        log_mel_spectrograms = tf.expand_dims(log_mel_spectrograms, 3)

        return log_mel_spectrograms

    def get_config(self):
        config = {
            'fft_size': self.fft_size,
            'hop_size': self.hop_size,
            'n_mels': self.n_mels,
            'sample_rate': self.sample_rate,
            'f_min': self.f_min,
            'f_max': self.f_max,
        }
        config.update(super(LogMelSpectrogram, self).get_config())

        return config